In [ ]:
import json
import urllib

from IPython.display import Image

# searching tools
from satsearch import Search
import geojsonio
from shapely.geometry import shape

# visualizing data tools
import rasterio
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
from rasterio.windows import Window
from rio_tiler.utils import get_vrt_transform
import numpy as np
from PIL import Image as PILImage

# Finding Imagery

Let's search for Landsat imagery over Yokohama. Development Seed [hosts an instance](https://sat-api.developmentseed.org/search/stac) of [`sat-api`](https://github.com/sat-utils/sat-api), an open source, [STAC](https://github.com/radiantearth/stac-spec) compliant API for searching for imagery.

In [ ]:
# AOI over Yokohama expressed as GeoJSON
aoi = {
    "type": "Polygon",
    "coordinates":[
        [
            [139.60,35.38],
            [139.69,35.38],
            [139.69,35.45],
            [139.60,35.45],
            [139.60,35.38]
        ]
    ]
}

In [ ]:
shape(aoi)

In [ ]:
# View GeoJSON on https://geojson.io
# This site can also be used for creating an AOI
geojsonio.display(json.dumps(aoi))

In [ ]:
# Choose other filters for our imagery
START_DATE = '2019-01-01T00:00:00Z'
END_DATE = '2019-05-01T23:59:59Z'
BOUNDING_BOX = shape(aoi).bounds
MIN_CLOUD_COVER = 0
MAX_CLOUD_COVER = 10

results = (
            Search(
                bbox=BOUNDING_BOX,
                time=f"{START_DATE}/{END_DATE}",
                query={
                    "eo:cloud_cover": {"gte": MIN_CLOUD_COVER, "lte": MAX_CLOUD_COVER},
                    "eo:sun_elevation": {"gt": 0},
                    "landsat:tier": {"eq": "T1"},
                    "collection": {"eq": "landsat-8-l1"},
                },
                sort=[
                    {"field": "eo:cloud_cover", "direction": "desc"},
                    {"field": "datetime", "direction": "asc"}
                ]
            )
            .items()
            .geojson()
        )

In [ ]:
# Show how many scenes were returned
len(results['features'])

In [ ]:
# Since the response is also GeoJSON, we can view the scene boundaries
geojsonio.display(json.dumps(results))

In [ ]:
# Show all the available dictionary keys on each scene/feature
first_scene = results['features'][0]
first_scene.keys()

In [ ]:
# Show all the available scene properties (metadata)
list(first_scene['properties'].keys())

In [ ]:
# Show all the available scene assets (data)
list(first_scene['assets'].keys())

## Downloading Imagery

Now let's download imagery. But instead of getting the whole file, let's save bandwidth and only grab the data we need. Because the files are organized as [Cloud-Optimized GeoTIFF](https://trac.osgeo.org/gdal/wiki/CloudOptimizedGeoTIFF)

In [ ]:
# View the scene thumbnail before downloading any large files
thumbnail = first_scene['assets']['thumbnail']
Image(thumbnail['href'])

In [ ]:
# The thumbnail is only 116 kB
r = urllib.request.urlopen(first_scene['assets']['thumbnail']['href'])
f'{r.info()["Content-Length"]} bytes'

In [ ]:
# One band of the actual data is 53.6 MB
red_band = first_scene['assets']['B4']['href']
r = urllib.request.urlopen(red_band)
f'{r.info()["Content-Length"]} bytes'

In [ ]:
# We can use rasterio to read the file headers and see how the data is organized before downloading
# Landsat files are arranged into easily downloadable square blocks which isn't true of GeoTIFF files in general
with rasterio.open(red_band) as src:
    print(src.block_shapes)
    
dg_open_url = 'https://opendata.digitalglobe.com/hurricane-florence/pre-event/2018-01-01/10400100380D1C00/0302213.tif'

with rasterio.open(dg_open_url) as src:
    print(src.block_shapes)

In [ ]:
def get_window(src, latlng_bbox):
    bounds = transform_bounds(CRS({ 'init': 'EPSG:4326' }), src.crs,  *latlng_bbox)
    transform, width, height = get_vrt_transform(src, bounds, dst_crs=src.crs)
    col_off = round((transform[2] - src.transform[2]) / src.transform[0])
    row_off = round((transform[5] - src.transform[5]) / src.transform[4])
    return Window(col_off=col_off, row_off=row_off, width=width, height=height)


with rasterio.open(red_band) as src:
    window = get_window(src, BOUNDING_BOX)
    print(window)
    array = src.read(1, window=window)

In [ ]:
def scale_array(arr):
    a_min = np.min(arr)
    a_max = np.max(arr)
    return ((arr - a_min) / (a_max - a_min) * 255).astype(np.uint8)

scaled_array = scale_array(array)
PILImage.fromarray(scaled_array * 4)

In [ ]:
bands = ['B4', 'B3', 'B2']
band_data = []
for band in bands:
    band_url = first_scene['assets'][band]['href']
    with rasterio.open(band_url) as band_src:
        window = get_window(band_src, BOUNDING_BOX)
        array = band_src.read(1, window=window)
        band_data.append(array)    

In [ ]:
scaled_bands = [scale_array(a) * 4 for a in band_data]
PILImage.fromarray(np.stack(scaled_bands, axis=2))